In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist, fashion_mnist

import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_train = x_train.reshape(-1, 784) / 255.0  
x_test = x_test.reshape(-1, 784) / 255.0

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
layer = model.layers[0]
len(layer.get_weights()[0][0])

256

In [8]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
938/938 [==============================] - 9s 7ms/step - loss: 0.2609 - accuracy: 0.9222 - val_loss: 0.1298 - val_accuracy: 0.9593
Epoch 2/5
938/938 [==============================] - 5s 6ms/step - loss: 0.0983 - accuracy: 0.9697 - val_loss: 0.1008 - val_accuracy: 0.9702
Epoch 3/5
938/938 [==============================] - 5s 6ms/step - loss: 0.0684 - accuracy: 0.9785 - val_loss: 0.0880 - val_accuracy: 0.9704
Epoch 4/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0493 - accuracy: 0.9845 - val_loss: 0.0765 - val_accuracy: 0.9765
Epoch 5/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0416 - accuracy: 0.9865 - val_loss: 0.0753 - val_accuracy: 0.9776


In [9]:
len(model.get_weights()[0][0])                                                 

256

In [10]:
def prune_weights(model, threshold):
    pruned_model = tf.keras.models.clone_model(model)
    pruned_model.set_weights(model.get_weights())  # Copy the original weights

    # Prune the weights below the threshold
    for layer in pruned_model.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            weights = layer.get_weights()
            weights[0][weights[0] < threshold] = 0.0  # Prune weights below threshold
            layer.set_weights(weights)
    
    return pruned_model


In [11]:

thresholds = [ -0.1, -0.2, -0.3, -0.5, -0.7,-1,0.0, 0.1, 0.2, 0.3, 0.5, 0.7,1]
performance = []

In [12]:
for threshold in thresholds:
    pruned_model = prune_weights(model, threshold)
    pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    pruned_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))
    loss, accuracy = pruned_model.evaluate(x_test, y_test)
    performance.append(accuracy)

Epoch 1/5
938/938 [==============================] - 13s 12ms/step - loss: 0.7513 - accuracy: 0.9307 - val_loss: 0.1038 - val_accuracy: 0.9683
Epoch 2/5
938/938 [==============================] - 7s 7ms/step - loss: 0.0682 - accuracy: 0.9796 - val_loss: 0.0841 - val_accuracy: 0.9746
Epoch 3/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0465 - accuracy: 0.9857 - val_loss: 0.0862 - val_accuracy: 0.9723
Epoch 4/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0383 - accuracy: 0.9876 - val_loss: 0.0901 - val_accuracy: 0.9732
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0959 - accuracy: 0.9732
Epoch 1/5
938/938 [==============================] - 9s 7ms/step - loss: 0.0628 - accuracy: 0.9800 - val_loss: 0.0835 - val_accuracy: 0.9736
Epoch 2/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0364 - accuracy: 0.9882 - val_loss: 0.0847 - val_accuracy: 0.9748
Epoch 3/5
938/938 [==============================] - 

In [13]:
acc_on_t = []

for threshold, acc in zip(thresholds, performance):
    ind = []
    ind.append(f'Threshold: {threshold}, Accuracy: {acc}')
    acc_on_t.append(ind)
print(acc_on_t)

[['Threshold: -0.1, Accuracy: 0.9732000231742859'], ['Threshold: -0.2, Accuracy: 0.9779999852180481'], ['Threshold: -0.3, Accuracy: 0.979200005531311'], ['Threshold: -0.5, Accuracy: 0.9787999987602234'], ['Threshold: -0.7, Accuracy: 0.9789000153541565'], ['Threshold: -1, Accuracy: 0.9768000245094299'], ['Threshold: 0.0, Accuracy: 0.9753000140190125'], ['Threshold: 0.1, Accuracy: 0.9749000072479248'], ['Threshold: 0.2, Accuracy: 0.9785000085830688'], ['Threshold: 0.3, Accuracy: 0.9641000032424927'], ['Threshold: 0.5, Accuracy: 0.9555000066757202'], ['Threshold: 0.7, Accuracy: 0.930400013923645'], ['Threshold: 1, Accuracy: 0.9424999952316284']]
